In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pkg_resources import resource_filename

from climpyrical.gridding import scale_model_obs
from climpyrical.data import read_data
from climpyrical.cmd.find_matched_model_vals import add_model_values

In [2]:
# Parameters cell

In [3]:
# Parameters
name = "PAnn"
station_dv = "annual_pr (mm)"
station_path = "data/station_inputs/pr_annual_mean_doy_MSC_25yr_for_maps.csv"
input_model_path = "data/model_inputs/pr_CanRCM4-LE_ens35_1951-2016_ann_sum_ensmean.nc"
medians = {"value": 892, "action": "multiply"}
fill_glaciers = True
output_notebook_path = "/data/results/intermediate/notebooks/"
preprocessed_model_path = "/data/results/intermediate/preprocessed_netcdf/"
preprocessed_stations_path = "/data/results/intermediate/preprocessed_stations/"
output_reconstruction_path = "/data/results/netcdf/"
output_tables_path = "/data/results/TableC2/"
output_figure_path = "/data/results/figures/"
mask_path = "data/masks/canada_mask_rp.nc"
north_mask_path = "data/masks/canada_mask_north_rp.nc"
nbcc_loc_path = "data/station_inputs/NBCC_2020_new_coords.xlsm"


Load NBCC data and fix trouble columns

In [4]:
nrc_path = resource_filename("climpyrical", nbcc_loc_path)
df_nrc = pd.read_excel(nrc_path).iloc[:-1]

# fill problem values with better values from 2015
id_typo = df_nrc[(df_nrc['2020 Longitude'] > 0) | (df_nrc['2020 Latitude'] < 40)].index
df_nrc.loc[id_typo, '2020 Longitude'] = df_nrc['2015 Long.'].values[id_typo]
df_nrc.loc[id_typo, '2020 Latitude'] = df_nrc['2015 Lat.'].values[id_typo]

# process to get the model values
df_nrc.rename(columns={"2020 Longitude": "lon",
                      "2020 Latitude": "lat"})

,LocationID,SortOrder,Location,Prov,2015 Elev (m),2020 Elev (m),Was Elev. Changed,2015 Lat.,2015 Long.,lat,lon,Was Lat. Changed,Was Long. Changed
0,1.0,10.0,100 Mile House,BC,1040,926,Yes,51.6500,-121.2833,51.64,-121.3,Yes,Yes
1,2.0,20.0,Abbotsford,BC,70,70,No,49.0517,-122.3289,49.05,-122.33,Yes,Yes
2,3.0,30.0,Agassiz,BC,15,25,Yes,49.2300,-121.7700,49.24,-121.76,Yes,Yes
3,4.0,40.0,Alberni,BC,12,50,Yes,49.2700,-124.8000,49.27,-124.8,No,No
4,5.0,50.0,Ashcroft,BC,305,305,No,50.7200,-121.2800,50.72,-121.28,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,632.0,6760.0,Kugluktuk,NU,10,15,Yes,67.8300,-115.0800,67.83,-115.1,No,Yes
676,637.0,6770.0,Nottingham Island,NU,30,30,No,63.1000,-78.0000,63.12,-77.93,Yes,Yes
677,639.0,6780.0,Resolute,NU,25,20,Yes,74.6800,-94.9000,74.70,-94.83,Yes,Yes
678,640.0,6790.0,Resolution Island,NU,5,0,Yes,61.3000,-64.8833,61.31,-64.87,Yes,Yes


In [5]:
# write to new dataframe
df_nrc_matched = pd.DataFrame(
    {
     'Location': df_nrc.Location, 
     'Prov': df_nrc.Prov,
     "2020 Elev (m)": df_nrc["2020 Elev (m)"],
     'lon': df_nrc['2020 Longitude'], 
     'lat': df_nrc['2020 Latitude']
     }
    
)
df_nrc_matched

,Location,Prov,2020 Elev (m),lon,lat
0,100 Mile House,BC,926,-121.3,51.64
1,Abbotsford,BC,70,-122.33,49.05
2,Agassiz,BC,25,-121.76,49.24
3,Alberni,BC,50,-124.8,49.27
4,Ashcroft,BC,305,-121.28,50.72
...,...,...,...,...,...
675,Kugluktuk,NU,15,-115.1,67.83
676,Nottingham Island,NU,30,-77.93,63.12
677,Resolute,NU,20,-94.83,74.70
678,Resolution Island,NU,0,-64.87,61.31


In [6]:
matched_df_nrc = pd.read_csv(
    resource_filename(
        "climpyrical",
        f"{preprocessed_stations_path}{name}.csv"
    ), 
    index_col=False
)

df_out = add_model_values(
    model_path = resource_filename(
        "climpyrical",
        f"{output_reconstruction_path}{name}_reconstruction.nc"
    ),
    df=df_nrc_matched,
    model_dv = station_dv
)

df_out.round(3).to_csv(
    resource_filename("climpyrical", f"{output_tables_path}{name}_TableC2.csv"), 
    index=False
)

INFO:root:Detect units: mm


INFO:root:rlat or rlon not detected in input file.converting assumes WGS84 coords to rotated pole


INFO:root:Matching coordinates now


INFO:root:Locating corresponding model valuesInterpolating to nearest if matched model value is NaN


In [7]:
df_out

,Location,Prov,2020 Elev (m),lon,lat,rlat,rlon,irlat,irlon,annual_pr (mm)
0,100 Mile House,BC,926,-121.3,51.64,6.472805,-14.893575,801,438,477.259092
1,Abbotsford,BC,70,-122.33,49.05,4.215381,-16.329523,750,406,1705.250993
2,Agassiz,BC,25,-121.76,49.24,4.279140,-15.914789,751,415,1895.883591
3,Alberni,BC,50,-124.8,49.27,4.957228,-17.785699,766,373,1689.128686
4,Ashcroft,BC,305,-121.28,50.72,5.592810,-15.163711,781,432,363.697212
...,...,...,...,...,...,...,...,...,...,...
675,Kugluktuk,NU,15,-115.1,67.83,21.173542,-7.222514,1132,611,221.571247
676,Nottingham Island,NU,30,-77.93,63.12,16.711316,8.872319,1032,975,408.444561
677,Resolute,NU,20,-94.83,74.70,27.208988,0.643710,1269,789,140.667118
678,Resolution Island,NU,0,-64.87,61.31,17.032368,15.487842,1039,1124,545.773601
